# Análise da Atividade Sobre Conceitos Introdutórios da Linguagem de Programação C

Professor(a), com o uso das técnicas de LA, pudemos responder algumas questões para essa etapa. Entre elas:

1. Como podemos identificar o grupo de alunos que estão com dificuldades, e quais o principais erros comentidos por esse alunos durante as atividade de programação? 

Para respondermos essa questão, no nosso exemplo, reunimos os dados extraidos do Banco de Dados da Plataforma Moodle tabela mdl_vpl_submissions, essa tabela continha dados referentes a cada questão das atividades (identificação do usuário, data de entrega, tempo de edição, quantidade de execuções, e nota). Assim, tratamos esses dados usando um Jupyter NoteBook, no qual agrupamos as tentativas de respostas para cada questão de acordo com o aluno, retiramos informações desnecessárias para a análise, além disso, combinamos o DataFrame gerado com os DataFrames das atividades anteriores. Desse modo, analisamos os resultados de 33 alunos, que totalizaram 4597 (quatro mil quinhentos e noventa e sete execuções de código). 

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go


df = pd.read_csv('vpl_01_limpo_1.csv')

df

,ID,VPL,USERID,DATESSUBMITTED,COMMENTS,DATEGRADED,GRADE,DATA_ENVIO,DATA_AVALIACAO,NOME_ALUNO
0,136,40,2,1679343696,NaN,0,NaN,2023-03-20 20:21:36,1970-01-01 00:00:00,NaN
1,138,40,2,1679343755,NaN,1679343829,1.000.000,2023-03-20 20:22:35,2023-03-20 20:23:49,NaN
2,139,41,2,1679344225,NaN,0,NaN,2023-03-20 20:30:25,1970-01-01 00:00:00,NaN
3,140,41,2,1679344267,NaN,0,NaN,2023-03-20 20:31:07,1970-01-01 00:00:00,NaN
4,141,42,2,1679344716,NaN,0,NaN,2023-03-20 20:38:36,1970-01-01 00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...
6498,27384,55,56,1686832373,NaN,1686832404,1.000.000,2023-06-15 12:32:53,2023-06-15 12:33:24,aluno 02
6499,27385,61,56,1686832954,NaN,0,NaN,2023-06-15 12:42:34,1970-01-01 00:00:00,aluno 02
6500,27386,61,56,1686832962,NaN,1686832967,1.000.000,2023-06-15 12:42:42,2023-06-15 12:42:47,aluno 02
6501,27414,68,65,1686919294,NaN,0,NaN,2023-06-16 12:41:34,1970-01-01 00:00:00,aluno 08


O código abaixo, representa a análise de desempenho dos alunos. Desse modo, o gráfico resume as tentativas de resposta para cada questão de acordo com o aluno, as linhas representam alunos individuais, as colunas representam as questões especificas, e a escala de cores que varia do amarelo ao azul ilustra a frequência de tentativas de envio, sendo que as cores claras (menos tentativas), indicam que os alunos conseguiram desenvolver os códigos de programação com poucas tentativas. Assim, as cores indicaram a questão era mais fácil ou que os alunos estavam bem-preparados. Por outro lado, as cores mais escuras (mais tentativas) indicando que os alunos precisaram de mais tentativas para resolver a questão, ou porque era mais difícil ou que os alunos tiveram dificuldades com o conceito abordado. Isso nos forneceu valiosos insights sobre o padrão de desempenho e o comportamento dos alunos durante o desenvolvimento das atividades de programação. 

In [3]:
# Passo 2: Use o método pivot para reorganizar os dados
pivot_table = df.pivot('NOME', 'NAME', 'CONTAGEM')


# Passo 3: Crie o heatmap com seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, cmap="YlGnBu", cbar=True)
plt.title('Heatmap de Contagem por Aluno e Questão')
plt.show()

/var/folders/cd/qj686nc9301drth50z2v1p8m0000gn/T/ipykernel_80258/282928690.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  pivot_table = df.pivot('NOME', 'NAME', 'CONTAGEM')


KeyError: 'NOME'

In [ ]:
# Passo 2: Agrupar por Aluno e somar as Contagens
total_envios = df.groupby('NOME')['CONTAGEM'].sum().reset_index()

# Passo 3: Ordenar os alunos pelo total de envios em ordem crescente
total_envios = total_envios.sort_values(by='CONTAGEM', ascending=True)

# Passo 4: Converter o DataFrame resultante em uma lista de tuplas (Aluno, Contagem)
lista_envios = total_envios.to_records(index=False).tolist()

print(lista_envios)

No código abaixo, realizamos a clusterização dos resultados usando para isso o algoritmo K-Means sobre o conjunto de dados coletados com base nas características da extraídas do dataframe da atividade, sendo elas: 'Aluno, 'Questão', e ‘Contagem Envios’. Essa clusterização nos forneceu informações sobre o comportamento dos alunos na atividade, com base nos atributos apresentado anteriormente.

In [ ]:
from sklearn.cluster import KMeans

# Passo 3: Aplicar o K-means para clusterizar em 3 grupos
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(total_envios[['CONTAGEM']])
total_envios['Cluster'] = kmeans.labels_

# Passo 4: Mostrar os resultados
print(total_envios)

# Passo 5: Visualização dos clusters
plt.scatter(total_envios['NOME'], total_envios['CONTAGEM'], c=total_envios['Cluster'], cmap='viridis')
plt.xlabel('NOME')
plt.ylabel('CONTAGEM')
plt.title('Clusters de Alunos Baseados na Contagem de Envios')
plt.show()


In [ ]:
import plotly.express as px

# Passo 3: Aplicar o K-means para clusterizar em 3 grupos
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(total_envios[['CONTAGEM']])
total_envios['Cluster'] = kmeans.labels_

# Passo 4: Criar o gráfico com plotly.express
fig = px.scatter(total_envios, x='NOME', y='CONTAGEM', color='Cluster', 
                 title='Clusters de Alunos Baseados na Contagem de Envios',
                 labels={'Aluno': 'NOME', 'CONTAGEM': 'CONTAGEM'},
                 color_continuous_scale=px.colors.sequential.Viridis)

fig.show()

In [ ]:
contagem_total = df['CONTAGEM'].sum()

print(f'A contagem total de envios é: {contagem_total}')